In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,accuracy_score
from scipy.stats import multivariate_normal
import pandas as pd

In [2]:
d = load_digits(as_frame=True)  #loading the pre-defined dataset.
#print(d.data.shape)
#print(d.target.shape)
#print(d.data[0])
d_frame = d.frame
print(d_frame)

      pixel_0_0  pixel_0_1  pixel_0_2  pixel_0_3  pixel_0_4  pixel_0_5  \
0           0.0        0.0        5.0       13.0        9.0        1.0   
1           0.0        0.0        0.0       12.0       13.0        5.0   
2           0.0        0.0        0.0        4.0       15.0       12.0   
3           0.0        0.0        7.0       15.0       13.0        1.0   
4           0.0        0.0        0.0        1.0       11.0        0.0   
...         ...        ...        ...        ...        ...        ...   
1792        0.0        0.0        4.0       10.0       13.0        6.0   
1793        0.0        0.0        6.0       16.0       13.0       11.0   
1794        0.0        0.0        1.0       11.0       15.0        1.0   
1795        0.0        0.0        2.0       10.0        7.0        0.0   
1796        0.0        0.0       10.0       14.0        8.0        1.0   

      pixel_0_6  pixel_0_7  pixel_1_0  pixel_1_1  ...  pixel_6_7  pixel_7_0  \
0           0.0        0.0      

In [3]:
print(d.data.shape)

(1797, 64)


In [4]:
print(d.target.shape)

(1797,)


In [5]:
help(multivariate_normal)

Help on multivariate_normal_gen in module scipy.stats._multivariate object:

class multivariate_normal_gen(multi_rv_generic)
 |  multivariate_normal_gen(seed=None)
 |  
 |  A multivariate normal random variable.
 |  
 |  The `mean` keyword specifies the mean. The `cov` keyword specifies the
 |  covariance matrix.
 |  
 |  Methods
 |  -------
 |  ``pdf(x, mean=None, cov=1, allow_singular=False)``
 |      Probability density function.
 |  ``logpdf(x, mean=None, cov=1, allow_singular=False)``
 |      Log of the probability density function.
 |  ``cdf(x, mean=None, cov=1, allow_singular=False, maxpts=1000000*dim, abseps=1e-5, releps=1e-5)``
 |      Cumulative distribution function.
 |  ``logcdf(x, mean=None, cov=1, allow_singular=False, maxpts=1000000*dim, abseps=1e-5, releps=1e-5)``
 |      Log of the cumulative distribution function.
 |  ``rvs(mean=None, cov=1, size=1, random_state=None)``
 |      Draw random samples from a multivariate normal distribution.
 |  ``entropy()``
 |      Comp

In [6]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, **options)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. If ``train_size`` is also Non

In [7]:
# # Splitting the dataset into the Training set and Test set
x_train, x_test = train_test_split(d_frame, test_size = 0.3, random_state = 0,stratify=d_frame[['target']])


In [8]:
help(np.cov)

Help on function cov in module numpy:

cov(m, y=None, rowvar=True, bias=False, ddof=None, fweights=None, aweights=None)
    Estimate a covariance matrix, given data and weights.
    
    Covariance indicates the level to which two variables vary together.
    If we examine N-dimensional samples, :math:`X = [x_1, x_2, ... x_N]^T`,
    then the covariance matrix element :math:`C_{ij}` is the covariance of
    :math:`x_i` and :math:`x_j`. The element :math:`C_{ii}` is the variance
    of :math:`x_i`.
    
    See the notes for an outline of the algorithm.
    
    Parameters
    ----------
    m : array_like
        A 1-D or 2-D array containing multiple variables and observations.
        Each row of `m` represents a variable, and each column a single
        observation of all those variables. Also see `rowvar` below.
    y : array_like, optional
        An additional set of variables and observations. `y` has the same form
        as that of `m`.
    rowvar : bool, optional
        If 

In [9]:
#calculating mean, covariance and prior for all class
mean_lst = []
cov_lst = []
prior_lst = []
for i in range(10):
    x=x_train[x_train['target']==i]
    print(x.shape)
    cov_0 = x.cov()
    cov_0 = np.array(cov_0.values)[:64,:64]
    x = x.to_numpy()
    mean = np.mean(x,axis=0)[:64]
    mean_lst.append(mean)
    cov_lst.append(cov_0)
    prior_lst.append(len(x)/len(x_train))

(124, 65)
(127, 65)
(124, 65)
(128, 65)
(127, 65)
(127, 65)
(127, 65)
(125, 65)
(122, 65)
(126, 65)


In [10]:
#predicting the value using Bayes classifier for test data and finding the accuracy 
y_pred_bayes = []
X_test = x_test.to_numpy()
for i in range(len(X_test)):
    Y = []
    for j in range(10):
        y = prior_lst[j]*(multivariate_normal.pdf(X_test[i,:64] , mean_lst[j] , cov_lst[j] , allow_singular=True))
        Y.append(y)
    y_pred_bayes.append(np.argmax(np.array(Y)))


print("Accuracy =",accuracy_score(X_test[:,64], y_pred_bayes)*100)

# To display the Confusion matrix
print(confusion_matrix(X_test[:,64], y_pred_bayes))   

Accuracy = 94.25925925925925
[[54  0  0  0  0  0  0  0  0  0]
 [ 0 49  0  0  0  1  0  0  5  0]
 [ 0  1 47  0  0  0  0  0  5  0]
 [ 0  0  0 50  0  0  0  0  1  4]
 [ 1  0  0  0 48  0  0  2  2  1]
 [ 0  0  0  2  0 52  0  0  1  0]
 [ 0  0  0  0  0  1 53  0  0  0]
 [ 0  0  0  0  1  0  0 52  0  1]
 [ 0  0  0  1  0  0  0  0 51  0]
 [ 0  0  0  0  0  0  0  0  1 53]]
